In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import catboost as ctb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings, math
warnings.filterwarnings('ignore')

from collections import Counter
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.linear_model import BayesianRidge, LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

from gplearn.genetic import SymbolicRegressor, SymbolicTransformer
from tqdm import tqdm_notebook
from gensim.models import Word2Vec

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

cv_dict = {}
cv_dict['0.8904'] = 0.8669
cv_dict['0.8969'] = 0.8729
cv_dict['0.9054_0.9052'] = 0.8814
cv_dict['0.9107_0.9104'] = 0.8884
cv_dict['0.9268_0.9267'] = 0.8891
cv_dict['0.9213_0.9212'] = 0.8897
cv_dict['0.9267_0.9266'] = 0.8923
cv_dict['0.9358_0.9358'] = 0.8931
cv_dict['0.9318_0.9317'] = 0.8938
cv_dict['0.9317_0.9317'] = 0.8957
cv_dict['0.9334_0.9333'] = 0.8994
cv_dict['0.9341_0.9342'] = 0.9008
cv_dict['0.9348_0.9346'] = 0.9032
cv_dict['0.9360_0.9359'] = 0.9055
cv_dict['0.9368_0.9368'] = 0.9060
cv_dict['0.9372_0.9373'] = 0.9063
cv_dict['0.9389_0.9375'] = 0.9065
cv_dict['0.9378_0.9377'] = 0.9076
cv_dict['0.9375_0.9375'] = 0.9080

# B榜
cv_dict['0.9378_0.9377'] = 0.9102

In [2]:
train = pd.read_csv('train_data.csv')
train['money_per_area'] = (train['tradeMoney'] / train['area']).astype(int)
train['room'] = [int(ht[0]) for ht in train['houseType']]

train = train[train['tradeMoney'] <= 22000]
train = train[train['region'] != 'RG00015']
train = train[train['tradeMoney'] >= 500]
train = train[(train['area'] >= 5) & (train['area'] <= 200)]
train = train[(train['totalFloor'] >= 1) & (train['totalFloor'] <= 60)]
train = train[(train['plate'] != 'BK00032') & (train['plate'] != 'BK00058')]
train.loc[(train['tradeMoney'] <= 1900) & (train['area'] >= 50), 'rentType'] = '合租'
train.loc[train['rentType'] == '--', 'rentType'] = '整租'
train.loc[train['tradeMoney'] >= 14000, 'rentType'] = '整租'
train.loc[(train['tradeMoney'] >= 9000) & (train['tradeMoney'] <= 14000) & (train['money_per_area'] >= 120), 'rentType'] = '整租'
train.loc[(train['tradeMoney'] <= 2000) & (train['room'] >= 3) & (train['rentType'] == '未知方式'), 'rentType'] = '合租'
train.loc[(train['houseType'] == '1室0厅0卫') & (train['rentType'] == '未知方式'), 'rentType'] = '整租'
train.loc[(train['houseType'] == '1室1厅1卫') & (train['tradeMoney'] >= 7000) & (train['rentType'] == '未知方式'), 'rentType'] = '整租'
train.loc[(train['tradeMoney'] >= 10000) & (train['room'] <= 1), 'rentType'] = '整租'
train.loc[(train['tradeMoney'] >= 9000) & (train['tradeMoney'] < 10000) & (train['room'] <= 1), 'rentType'] = '整租'
train = train.drop(['money_per_area', 'room'], axis=1)
# 实验
print(train.shape)

test = pd.read_csv('test_b.csv')
test.loc[test['houseDecoration'] == '毛坯', 'rentType'] = '整租'

train['type'] = 1
test['type'] = 0
df = pd.concat([train, test], ignore_index=True)[train.columns]

(40586, 51)


In [3]:
train['rentType'].value_counts()

# 30329
# 5382
# 5201

未知方式    27227
合租       7139
整租       6220
Name: rentType, dtype: int64

In [4]:
def rentType_trans(rt):
    if rt == '未知方式':
        return 0
    elif rt == '整租':
        return 1
    elif rt == '合租':
        return 2
    else:
        return 1

df['rentType'] = df['rentType'].apply(rentType_trans)

df['room'] = [int(ht[0]) for ht in df['houseType']]
df['hall'] = [int(ht[2]) for ht in df['houseType']]
df['bath'] = [int(ht[-2]) for ht in df['houseType']]

df.loc[(df['area'] <= 50) & (df['room'] >= 3) & (df['rentType'] == 0), 'rentType'] = 2
df.loc[(df['rentType'] == 2) & (df['area'] > 50), 'area'] = df['area'] / (df['room'] + 1)

def houseFloor_trans(hf):
    if hf == '低':
        return 0
    elif hf == '中':
        return 1
    else:
        return 2

df['houseFloor'] = df['houseFloor'].apply(houseFloor_trans)

def houseToward_trans(ht):
    if ht == '暂无数据':
        return 0
    elif ht in ['南', '南北', '北']:
        return 1
    elif ht in ['东', '东西', '西']:
        return 2
    elif ht in ['西北', '东南']:
        return 3
    elif ht in ['东北', '西南']:
        return 4
    
df['houseToward'] = df['houseToward'].apply(houseToward_trans)

def houseDecoration_trans(hd):
    if hd == '其他':
        return 0
    elif hd == '毛坯':
        return 1
    elif hd == '简装':
        return 2
    else:
        return 3

df['houseDecoration'] = df['houseDecoration'].apply(houseDecoration_trans)
df.loc[(df['houseDecoration'] == 1) & (df['rentType'] == 0), 'rentType'] = 1

df['communityName'] = [int(cn[2: ]) for cn in df['communityName']]
df['region'] = [int(re[2: ]) for re in df['region']]
df['plate'] = [int(pl[2: ]) for pl in df['plate']]

df['buildYear'] = [np.nan if by == '暂无信息' else float(by) for by in df['buildYear']]
df['trade_month'] = [int(time.split('/')[1]) for time in df['tradeTime']]
df['season'] = [int(np.ceil(month / 3)) for month in df['trade_month']]

df = df.drop(['city'], axis=1)

In [5]:
df['totalRoom'] = df['room'] + df['hall'] + df['bath']
df['area_per_room'] = df['area'] / df['totalRoom']
df['area_for_room'] = df['area'] / df['room']
df['floor_ratio'] = round(df['totalFloor'] * ((df['houseFloor'] * 2 + 1) / 6))
df['per_pv'] = df['pv'] / df['uv']
df['mean_pv'] = (df['pv'] + df['uv']) / 2
df['max_pv'] = np.max(df[['pv', 'uv']], axis=1)
df['min_pv'] = np.min(df[['pv', 'uv']], axis=1)
df['std_pv'] = np.std(df[['pv', 'uv']], axis=1)
df['worker_ratio'] = df['totalWorkers'] / df['residentPopulation']

In [6]:
df['room'] = [1 if rt == 2 else room for rt, room in np.array(df[['rentType', 'room']])]
df['totalRoom'] = [1 if rt == 2 else tr for rt, tr in np.array(df[['rentType', 'totalRoom']])]
# df['area'] = [area / room if rt == 2 and area >= 50 else area for rt, room, area in np.array(df[['rentType', 'room', 'area']])]

In [7]:
a = df.groupby(['plate', 'rentType']).mean().reset_index()[['plate', 'rentType', 'tradeMoney']].sort_values('tradeMoney', ascending=False).rename(columns={'tradeMoney': 'plate_mean'})
b = df.groupby(['region', 'rentType']).mean().reset_index()[['region', 'rentType', 'tradeMoney']].sort_values('tradeMoney', ascending=False).rename(columns={'tradeMoney': 'region_mean'})
c = df.groupby(['plate', 'rentType']).median().reset_index()[['plate', 'rentType', 'tradeMoney']].sort_values('tradeMoney', ascending=False).rename(columns={'tradeMoney': 'plate_median'})
d = df.groupby(['region', 'rentType']).median().reset_index()[['region', 'rentType', 'tradeMoney']].sort_values('tradeMoney', ascending=False).rename(columns={'tradeMoney': 'region_median'})

df = df.merge(a, how='left', on=['plate', 'rentType'])
df = df.merge(b, how='left', on=['region', 'rentType'])
df = df.merge(c, how='left', on=['plate', 'rentType'])
df = df.merge(d, how='left', on=['region', 'rentType'])

In [8]:
a = df.groupby(['region']).size().reset_index().rename(columns={0: 'region_num'})
df = df.merge(a, how='left', on=['region'])
a = df.groupby(['plate']).size().reset_index().rename(columns={0: 'plate_num'})
df = df.merge(a, how='left', on=['plate'])
a = df.groupby(['communityName']).size().reset_index().rename(columns={0: 'community_num'})
df = df.merge(a, how='left', on=['communityName'])
a = df[['region', 'plate']].groupby(['region']).apply(lambda x: x['plate'].nunique()).reset_index().rename(columns={0: 'plate_contain'})
df = df.merge(a, how='left', on=['region'])
a = df.groupby('plate').apply(lambda x: x['communityName'].nunique()).reset_index().rename(columns={0: 'community_contain'})
df = df.merge(a, how='left', on=['plate'])

In [9]:
a = df.groupby('plate').mean().reset_index()[['plate', 'totalFloor']].rename(columns={'totalFloor': 'plate_mean_floor'})
df = df.merge(a, how='left', on=['plate'])
a = df.groupby('plate').median().reset_index()[['plate', 'totalFloor']].rename(columns={'totalFloor': 'plate_median_floor'})
df = df.merge(a, how='left', on=['plate'])
a = df.groupby('plate').mean().reset_index()[['plate', 'area']].rename(columns={'area': 'plate_mean_area'})
df = df.merge(a, how='left', on=['plate'])
a = df.groupby('plate').median().reset_index()[['plate', 'area']].rename(columns={'area': 'plate_median_area'})
df = df.merge(a, how='left', on=['plate'])

In [10]:
df['room_ratio'] = df['room'] / df['totalRoom']
df['hall_ratio'] = df['hall'] / df['totalRoom']
df['bath_ratio'] = df['bath'] / df['totalRoom']
df['room-bath'] = df['room'] - df['bath']

df['max_type'] = np.argmax(np.array(df[['room', 'hall', 'bath']]), axis=1)

df['trade_avg'] = df['totalTradeArea'] / (df['tradeSecNum'] + 1)

df['originWorkers'] = df['totalWorkers'] - df['newWorkers']

# df['feature2'] = df['plate_mean'] / df['region_mean']

df['area_part'] = pd.qcut(df['area'], q=6, labels=[0, 1, 2, 3, 4, 5])

df['tf_part'] = pd.qcut(df['totalFloor'], q=8, duplicates='drop', labels=[i for i in range(6)])

In [11]:
# plate, room, area, totalRoom

a = df.groupby(['communityName']).apply(lambda x: x['totalRoom'].tolist()).reset_index()
a['mean_totalRoom'] = [np.mean(i) for i in a[0]]
a['median_totalRoom'] = [np.median(i) for i in a[0]]
a = a.drop(0, axis=1)
df = df.merge(a, how='left', on=['communityName'])

a = df.groupby(['communityName']).apply(lambda x: x['room'].tolist()).reset_index()
a['mean_room'] = [np.mean(i) for i in a[0]]
a['median_room'] = [np.median(i) for i in a[0]]
a = a.drop(0, axis=1)
df = df.merge(a, how='left', on=['communityName'])

a = df.groupby(['plate', 'area_part']).apply(lambda x: x['totalRoom'].tolist()).reset_index()
a['plate_mean_totalRoom'] = [np.mean(i) for i in a[0]]
a['plate_median_totalRoom'] = [np.median(i) for i in a[0]]
a = a.drop(0, axis=1)
df = df.merge(a, how='left', on=['plate', 'area_part'])

a = df.groupby(['plate', 'area_part']).apply(lambda x: x['room'].tolist()).reset_index()
a['plate_mean_room'] = [np.mean(i) for i in a[0]]
a['plate_median_room'] = [np.median(i) for i in a[0]]
a = a.drop(0, axis=1)
df = df.merge(a, how='left', on=['plate', 'area_part'])

In [12]:
for col in ['subwayStationNum', 'busStationNum', 'interSchoolNum', 'schoolNum', 'privateSchoolNum', 'hospitalNum',
       'drugStoreNum', 'gymNum', 'bankNum', 'shopNum', 'parkNum', 'mallNum', 'superMarketNum']:
    df['per_' + col] = df['residentPopulation'] / (df[col] + 1)
    df['workers_per_' + col] = df['totalWorkers'] / (df[col] + 1)

df['sub_div_bus'] = df['per_subwayStationNum'] / df['per_busStationNum']
df['sub_minus_bus'] = df['per_subwayStationNum'] - df['per_busStationNum']
df['sub_bus_ratio'] = df['sub_minus_bus'] / df['per_subwayStationNum']

df['workers_sub_div_bus'] = df['workers_per_subwayStationNum'] / df['workers_per_busStationNum']
df['workers_sub_minus_bus'] = df['workers_per_subwayStationNum'] - df['workers_per_busStationNum']
df['workers_sub_bus_ratio'] = df['workers_sub_minus_bus'] / df['workers_per_subwayStationNum']

count_feature_list = []
for i in ['totalFloor', 'tf_part']:
    count_feature_list.append('count_' + i)
    df['count_' + i] = df[i].map(df[i].value_counts())
    
a = df.groupby(['count_tf_part', 'count_totalFloor']).apply(lambda x: x['buildYear'].tolist()).reset_index()
a['is_nan'] = [np.sign(np.mean(i)) for i in a[0]]
a['is_nan'].fillna(0, inplace=True)
a['is_nan'] = 1 - a['is_nan']
a = a[a['is_nan'] == 1]
a['buildYear'] = [int(np.nanmean(i)) for i in a[0]]
a = a.drop([0, 'is_nan'], axis=1)

b = df[df['buildYear'].isna()][['count_tf_part', 'count_totalFloor']]
b_index = b.index
b = b.merge(a, how='left', on=['count_tf_part', 'count_totalFloor'])
b.index = b_index
df['buildYear'].fillna(b['buildYear'], axis='index', inplace=True)
df['used_year'] = 2018 - df['buildYear']
df['area-year'] = df['area'] / (df['used_year'] + 1)
    
for i in ['buildYear', 'room', 'hall', 'bath', 'area_part']:
    count_feature_list.append('count_' + i)
    df['count_' + i] = df[i].map(df[i].value_counts())

df['drug_per_hosp'] = df['drugStoreNum'] / (df['hospitalNum'] + 1)
df['supermarket_ratio'] = df['superMarketNum'] / df['shopNum']
df['new_with_subway'] = (df['subwayStationNum'] + 1) / (df['used_year']  + 1)
df['new_with_bus'] = df['busStationNum'] / (df['used_year'] + 1)
df['private_school_ratio'] = df['interSchoolNum'] / (df['privateSchoolNum'] + 1)
df['total_school'] = df['schoolNum'] + df['privateSchoolNum']

In [13]:
# rentType
a = df.groupby(['communityName']).apply(lambda x: x['rentType'].tolist()).reset_index()
a['0num'] = [single.count(0) for single in a[0]]
a['1num'] = [single.count(1) for single in a[0]]
a['2num'] = [single.count(2) for single in a[0]]
a['0ratio'] = a['0num'] / np.sum(a[['0num', '1num', '2num']], axis=1)
a['1ratio'] = a['1num'] / np.sum(a[['0num', '1num', '2num']], axis=1)
a['2ratio'] = a['2num'] / np.sum(a[['0num', '1num', '2num']], axis=1)
a = a[['communityName', '0num', '1num', '2num', '0ratio', '1ratio', '2ratio']]
a['most_type'] = np.argmax(a[['0num', '1num', '2num']].values, axis=1)
df = df.merge(a, how='left', on=['communityName'])

# -------------------------------------------------------------------------------------

# a = df.groupby(['plate']).apply(lambda x: x['rentType'].tolist()).reset_index()
# a['0pnum'] = [single.count(0) for single in a[0]]
# a['1pnum'] = [single.count(1) for single in a[0]]
# a['2pnum'] = [single.count(2) for single in a[0]]
# a['0pratio'] = a['0pnum'] / np.sum(a[['0pnum', '1pnum', '2pnum']], axis=1)
# a['1pratio'] = a['1pnum'] / np.sum(a[['0pnum', '1pnum', '2pnum']], axis=1)
# a['2pratio'] = a['2pnum'] / np.sum(a[['0pnum', '1pnum', '2pnum']], axis=1)
# a = a[['plate', '0pnum', '1pnum', '2pnum', '0pratio', '1pratio', '2pratio']]
# df = df.merge(a, how='left', on=['plate'])

In [14]:
# area
a = df.groupby(['communityName']).apply(lambda x: x['area'].tolist()).reset_index()
a['comm_area_len'] = [len(a) for a in a[0]]
a = a.merge(df.groupby(['communityName']).apply(lambda x: x['area'].nunique()).reset_index().rename(columns={0: 'comm_area_kind'}), how='left', on=['communityName'])
a['comm_mean_area'] = [np.mean(i) for i in a[0]]
a['comm_median_area'] = [np.median(i) for i in a[0]]
a['comm_max_area'] = [np.max(i) for i in a[0]]
a['comm_min_area'] = [np.min(i) for i in a[0]]
a['comm_std_area'] = [np.std(i) for i in a[0]]
a['comm_area_common_num'] = a['comm_area_len'] - a['comm_area_kind']
a['comm_area_unique_ratio'] = a['comm_area_kind'] / a['comm_area_len']
a['comm_area_common_ratio'] = 1 - a['comm_area_kind'] / a['comm_area_len']
a = a.drop(0, axis=1)
df = df.merge(a, how='left', on=['communityName'])

# -----------------------------------------------------------------------------------------

# a = df.groupby(['plate']).apply(lambda x: x['area'].tolist()).reset_index()
# a['plate_area_len'] = [len(a) for a in a[0]]
# a = a.merge(df.groupby(['plate']).apply(lambda x: x['area'].nunique()).reset_index().rename(columns={0: 'plate_area_kind'}), how='left', on=['plate'])
# # a['plate_mean_area'] = [np.mean(i) for i in a[0]]
# # a['plate_median_area'] = [np.median(i) for i in a[0]]
# a['plate_max_area'] = [np.max(i) for i in a[0]]
# a['plate_min_area'] = [np.min(i) for i in a[0]]
# a['plate_std_area'] = [np.std(i) for i in a[0]]
# a['plate_area_common_num'] = a['plate_area_len'] - a['plate_area_kind']
# a['plate_area_unique_ratio'] = a['plate_area_kind'] / a['plate_area_len']
# a['plate_area_common_ratio'] = 1 - a['plate_area_kind'] / a['plate_area_len']
# a = a.drop(0, axis=1)
# df = df.merge(a, how='left', on=['plate'])

In [15]:
# houseDecoration
a = df.groupby('communityName').apply(lambda x: x['houseDecoration'].tolist()).reset_index()
a['comm_deco_len'] = [len(i) for i in a[0]]
a = a.merge(df.groupby(['communityName']).apply(lambda x: x['houseDecoration'].nunique()).reset_index().rename(columns={0: 'comm_deco_kind'}), how='left', on=['communityName'])
a['0deco'] = [single.count(0) for single in a[0]]
a['1deco'] = [single.count(1) for single in a[0]]
a['2deco'] = [single.count(2) for single in a[0]]
a['3deco'] = [single.count(3) for single in a[0]]
a['0deco_ratio'] = a['0deco'] / np.sum(a[['0deco', '1deco', '2deco', '3deco']], axis=1)
a['1deco_ratio'] = a['1deco'] / np.sum(a[['0deco', '1deco', '2deco', '3deco']], axis=1)
a['2deco_ratio'] = a['2deco'] / np.sum(a[['0deco', '1deco', '2deco', '3deco']], axis=1)
a['3deco_ratio'] = a['3deco'] / np.sum(a[['0deco', '1deco', '2deco', '3deco']], axis=1)
a = a.drop(0, axis=1)
df = df.merge(a, how='left', on=['communityName'])

# --------------------------------------------------------------------------------------------

# a = df.groupby('plate').apply(lambda x: x['houseDecoration'].tolist()).reset_index()
# a['plate_pdeco_len'] = [len(i) for i in a[0]]
# a = a.merge(df.groupby(['plate']).apply(lambda x: x['houseDecoration'].nunique()).reset_index().rename(columns={0: 'plate_pdeco_kind'}), how='left', on=['plate'])
# a['0pdeco'] = [single.count(0) for single in a[0]]
# a['1pdeco'] = [single.count(1) for single in a[0]]
# a['2pdeco'] = [single.count(2) for single in a[0]]
# a['3pdeco'] = [single.count(3) for single in a[0]]
# a['0pdeco_ratio'] = a['0pdeco'] / np.sum(a[['0pdeco', '1pdeco', '2pdeco', '3pdeco']], axis=1)
# a['1pdeco_ratio'] = a['1pdeco'] / np.sum(a[['0pdeco', '1pdeco', '2pdeco', '3pdeco']], axis=1)
# a['2pdeco_ratio'] = a['2pdeco'] / np.sum(a[['0pdeco', '1pdeco', '2pdeco', '3pdeco']], axis=1)
# a['3pdeco_ratio'] = a['3pdeco'] / np.sum(a[['0pdeco', '1pdeco', '2pdeco', '3pdeco']], axis=1)
# a = a.drop(0, axis=1)
# df = df.merge(a, how='left', on=['plate'])

In [16]:
# houseToward
a = df.groupby('communityName').apply(lambda x: x['houseToward'].tolist()).reset_index()
a = a.merge(df.groupby(['communityName']).apply(lambda x: x['houseToward'].nunique()).reset_index().rename(columns={0: 'comm_toward_kind'}), how='left', on=['communityName'])
a['to0'] = [i.count(0) for i in a[0]]
a['to1'] = [i.count(1) for i in a[0]]
a['to2'] = [i.count(2) for i in a[0]]
a['to3'] = [i.count(3) for i in a[0]]
a['to4'] = [i.count(4) for i in a[0]]
a['to0_ratio'] = a['to0'] / np.sum(a[['to0', 'to1', 'to2', 'to3', 'to4']], axis=1)
a['to1_ratio'] = a['to1'] / np.sum(a[['to0', 'to1', 'to2', 'to3', 'to4']], axis=1)
a['to2_ratio'] = a['to2'] / np.sum(a[['to0', 'to1', 'to2', 'to3', 'to4']], axis=1)
a['to3_ratio'] = a['to3'] / np.sum(a[['to0', 'to1', 'to2', 'to3', 'to4']], axis=1)
a['to4_ratio'] = a['to4'] / np.sum(a[['to0', 'to1', 'to2', 'to3', 'to4']], axis=1)
a['max_toward_type'] = np.argmax(a[['to0', 'to1', 'to2', 'to3', 'to4']].values, axis=1)
a['max_toward_ratio'] = np.max(a[['to0', 'to1', 'to2', 'to3', 'to4']], axis=1) / np.sum(a[['to0', 'to1', 'to2', 'to3', 'to4']], axis=1)
a = a.drop(0, axis=1)
df = df.merge(a, how='left', on=['communityName'])

# ---------------------------------------------------------------------------------------------------------------------------------------

# a = df.groupby('plate').apply(lambda x: x['houseToward'].tolist()).reset_index()
# a = a.merge(df.groupby(['plate']).apply(lambda x: x['houseToward'].nunique()).reset_index().rename(columns={0: 'plate_ptoward_kind'}), how='left', on=['plate'])
# a['pto0'] = [i.count(0) for i in a[0]]
# a['pto1'] = [i.count(1) for i in a[0]]
# a['pto2'] = [i.count(2) for i in a[0]]
# a['pto3'] = [i.count(3) for i in a[0]]
# a['pto4'] = [i.count(4) for i in a[0]]
# a['pto0_ratio'] = a['pto0'] / np.sum(a[['pto0', 'pto1', 'pto2', 'pto3', 'pto4']], axis=1)
# a['pto1_ratio'] = a['pto1'] / np.sum(a[['pto0', 'pto1', 'pto2', 'pto3', 'pto4']], axis=1)
# a['pto2_ratio'] = a['pto2'] / np.sum(a[['pto0', 'pto1', 'pto2', 'pto3', 'pto4']], axis=1)
# a['pto3_ratio'] = a['pto3'] / np.sum(a[['pto0', 'pto1', 'pto2', 'pto3', 'pto4']], axis=1)
# a['pto4_ratio'] = a['pto4'] / np.sum(a[['pto0', 'pto1', 'pto2', 'pto3', 'pto4']], axis=1)
# a['max_ptoward_type'] = np.argmax(a[['pto0', 'pto1', 'pto2', 'pto3', 'pto4']].values, axis=1)
# a['max_ptoward_ratio'] = np.max(a[['pto0', 'pto1', 'pto2', 'pto3', 'pto4']], axis=1) / np.sum(a[['pto0', 'pto1', 'pto2', 'pto3', 'pto4']], axis=1)
# a = a.drop(0, axis=1)
# df = df.merge(a, how='left', on=['plate'])

In [17]:
# totalFloor
a = df.groupby('communityName').apply(lambda x: x['totalFloor'].tolist()).reset_index()
a = a.merge(df.groupby(['communityName']).apply(lambda x: x['totalFloor'].nunique()).reset_index().rename(columns={0: 'comm_totalFloor_unique'}), how='left', on=['communityName'])
a['comm_totalFloor_len'] = [len(i) for i in a[0]]
a['majority'] = [np.argmax(np.bincount(i)) for i in a[0]]
a['majority_num'] = [np.max(np.bincount(i)) for i in a[0]]
a['majority_ratio'] = [np.max(np.bincount(i)) / len(i) for i in a[0]]
a['totalFloor_std'] = [np.std(i) for i in a[0]]
a = a.drop(0, axis=1)
df = df.merge(a, how='left', on=['communityName'])

# ---------------------------------------------------------------------------------------------------

# a = df.groupby('plate').apply(lambda x: x['totalFloor'].tolist()).reset_index()
# a = a.merge(df.groupby(['plate']).apply(lambda x: x['totalFloor'].nunique()).reset_index().rename(columns={0: 'plate_totalFloor_unique'}), how='left', on=['plate'])
# a['plate_totalFloor_len'] = [len(i) for i in a[0]]
# a['pmajority'] = [np.argmax(np.bincount(i)) for i in a[0]]
# a['pmajority_num'] = [np.max(np.bincount(i)) for i in a[0]]
# a['pmajority_ratio'] = [np.max(np.bincount(i)) / len(i) for i in a[0]]
# a['ptotalFloor_std'] = [np.std(i) for i in a[0]]
# a = a.drop(0, axis=1)
# df = df.merge(a, how='left', on=['plate'])

In [18]:
a = df.groupby('plate').apply(lambda x: x['buildYear'].tolist()).reset_index()
a['mean_year'] = [np.nanmean(i) for i in a[0]]
a['most_year'] = [Counter(i).most_common()[0][0] for i in a[0]]
a['most_count'] = [i.count(most) for i, most in a[[0, 'most_year']].values]
a['most_ratio'] = [mc / len(i) for mc, i in a[['most_count', 0]].values]
a = a.drop(0, axis=1)
df = df.merge(a, how='left', on=['plate'])

+ 越老越贵，越小越贵
+ 岗位密集区租金更高
+ 上班族越多，附近租金越贵
+ 地铁相比于公交来说，对多数需要远距离通勤的租客来说是首要考虑的交通工具，因此对房租的影响也最为显著
+ 每平方米月租金：租金/平方米

In [19]:
train = df[df['type'] == 1]
train['money_per_area'] = train['tradeMoney'] / train['area']
a = train.groupby('communityName').mean().reset_index()[['communityName', 'money_per_area']]
a['money_per_area'] = [int(i + (10 - i % 10)) if i % 10 >= 5 else int(i - i % 10) for i in a['money_per_area']]
df = df.merge(a, how='left', on=['communityName'])

In [20]:
a = df.groupby(['area_part', 'rentType']).apply(lambda x: x['money_per_area'].tolist()).reset_index()
a['area_type_mean'] = [np.nanmean(i) for i in a[0]]
a['area_type_median'] = [np.nanmedian(i) for i in a[0]]
a['area_type_max'] = [np.nanmax(i) for i in a[0]]
a['area_type_min'] = [np.nanmin(i) for i in a[0]]
a['area_type_num'] = [len(i) for i in a[0]]
a = a.drop(0, axis=1)
df = df.merge(a, how='left', on=['area_part', 'rentType'])

In [21]:
a = df.groupby('plate').apply(lambda x: x['money_per_area'].tolist()).reset_index()
a['mean_money'] = [np.nanmean(i) for i in a[0]]
a['median_money'] = [np.nanmedian(i) for i in a[0]]
a['std_money'] = [np.nanstd(i) for i in a[0]]
a = a.drop(0, axis=1)
df = df.merge(a, how='left', on=['plate'])

# a = df.groupby(['plate', 'rentType']).apply(lambda x: x['money_per_area'].tolist()).reset_index()
# a['plate_type_mean_money'] = [np.mean(i) for i in a[0]]
# a['plate_type_median_money'] = [np.median(i) for i in a[0]]
# a = a.drop(0, axis=1)
# df = df.merge(a, how='left', on=['plate', 'rentType'])

# a = df.groupby(['region', 'rentType']).apply(lambda x: x['money_per_area'].tolist()).reset_index()
# a['region_type_mean_money'] = [np.nanmean(i) for i in a[0]]
# a['region_type_median_money'] = [np.nanmedian(i) for i in a[0]]
# a = a.drop(0, axis=1)
# df = df.merge(a, how='left', on=['region', 'rentType'])

In [22]:
a = df.groupby(['area_part', 'rentType']).apply(lambda x: x['money_per_area'].tolist()).reset_index()
a['mean_area_part_money'] = [np.nanmean(i) for i in a[0]]
a['median_area_part_money'] = [np.nanmedian(i) for i in a[0]]
a['std_area_part_money'] = [np.nanstd(i) for i in a[0]]
a['max_area_part_money'] = [np.nanmax(i) for i in a[0]]
a['min_area_part_money'] = [np.nanmin(i) for i in a[0]]
a = a.drop(0, axis=1)
df = df.merge(a, how='left', on=['area_part', 'rentType'])

In [23]:
df['tradeTime'] = df['tradeTime'].astype('datetime64')
df['trade_day'] = df['tradeTime'].dt.day
df['dayofyear'] = df['tradeTime'].dt.dayofyear
df['weekofyear'] = df['tradeTime'].dt.weekofyear
df['weekday'] = df['tradeTime'].dt.dayofweek
df['weekend'] = [1 if wd == 5 or wd == 6 else 0 for wd in df['weekday']]

a = df.groupby(['trade_month', 'trade_day']).size().reset_index().rename(columns={0: 'day_count'})
df = df.merge(a, how='left', on=['trade_month', 'trade_day'])

In [24]:
ht_count_dict, ht_encode_dict = {}, {}
for i, ht in enumerate(df['houseType'].unique()):
    ht_count_dict[ht] = df['houseType'].value_counts()[ht]
    ht_encode_dict[ht] = i
    
df['ht_count'] = df['houseType'].apply(lambda x: ht_count_dict[x])
df['ht_encode'] = df['houseType'].apply(lambda x: ht_encode_dict[x])

df['sale_to_rent_ratio'] = df['tradeMeanPrice'] / df['money_per_area']

# 对室数量，厅数量，weekofyear做groupby

In [25]:
df.sample(10)

,ID,area,rentType,houseType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,region,plate,buildYear,saleSecHouseNum,subwayStationNum,busStationNum,interSchoolNum,schoolNum,privateSchoolNum,hospitalNum,drugStoreNum,gymNum,bankNum,shopNum,parkNum,mallNum,superMarketNum,totalTradeMoney,totalTradeArea,tradeMeanPrice,tradeSecNum,totalNewTradeMoney,totalNewTradeArea,tradeNewMeanPrice,tradeNewNum,remainNewNum,supplyNewNum,supplyLandNum,supplyLandArea,tradeLandNum,tradeLandArea,landTotalPrice,landMeanPrice,totalWorkers,newWorkers,residentPopulation,pv,uv,lookNum,tradeTime,tradeMoney,type,room,hall,bath,trade_month,season,totalRoom,area_per_room,area_for_room,floor_ratio,per_pv,mean_pv,max_pv,min_pv,std_pv,worker_ratio,plate_mean,region_mean,plate_median,region_median,region_num,plate_num,community_num,plate_contain,community_contain,plate_mean_floor,plate_median_floor,plate_mean_area,plate_median_area,room_ratio,hall_ratio,bath_ratio,room-bath,max_type,trade_avg,originWorkers,area_part,tf_part,mean_totalRoom,median_totalRoom,mean_room,median_room,plate_mean_totalRoom,plate_median_totalRoom,plate_mean_room,plate_median_room,per_subwayStationNum,workers_per_subwayStationNum,per_busStationNum,workers_per_busStationNum,per_interSchoolNum,workers_per_interSchoolNum,per_schoolNum,workers_per_schoolNum,per_privateSchoolNum,workers_per_privateSchoolNum,per_hospitalNum,workers_per_hospitalNum,per_drugStoreNum,workers_per_drugStoreNum,per_gymNum,workers_per_gymNum,per_bankNum,workers_per_bankNum,per_shopNum,workers_per_shopNum,per_parkNum,workers_per_parkNum,per_mallNum,workers_per_mallNum,per_superMarketNum,workers_per_superMarketNum,sub_div_bus,sub_minus_bus,sub_bus_ratio,workers_sub_div_bus,workers_sub_minus_bus,workers_sub_bus_ratio,count_totalFloor,count_tf_part,used_year,area-year,count_buildYear,count_room,count_hall,count_bath,count_area_part,drug_per_hosp,supermarket_ratio,new_with_subway,new_with_bus,private_school_ratio,total_school,0num,1num,2num,0ratio,1ratio,2ratio,most_type,comm_area_len,comm_area_kind,comm_mean_area,comm_median_area,comm_max_area,comm_min_area,comm_std_area,comm_area_common_num,comm_area_unique_ratio,comm_area_common_ratio,comm_deco_len,comm_deco_kind,0deco,1deco,2deco,3deco,0deco_ratio,1deco_ratio,2deco_ratio,3deco_ratio,comm_toward_kind,to0,to1,to2,to3,to4,to0_ratio,to1_ratio,to2_ratio,to3_ratio,to4_ratio,max_toward_type,max_toward_ratio,comm_totalFloor_unique,comm_totalFloor_len,majority,majority_num,majority_ratio,totalFloor_std,mean_year,most_year,most_count,most_ratio,money_per_area,area_type_mean,area_type_median,area_type_max,area_type_min,area_type_num,mean_money,median_money,std_money,mean_area_part_money,median_area_part_money,std_area_part_money,max_area_part_money,min_area_part_money,trade_day,dayofyear,weekofyear,weekday,weekend,day_count,ht_count,ht_encode,sale_to_rent_ratio
6402,100303946,47.000000,0,1室1厅1卫,2,6,1,0,3562,12,9,1995.0,0,2,56,0,53,8,4,34,16,23,154,4,1,37,520865400,12190.94,42725.61427,221,0,0,0.00000,0,214,0,0,0.0,0,0.0,0,0.0,4420,196,165293,166.0,47.0,0,2018-11-08,3200.0,1,1,1,1,11,4,3,15.666667,47.000000,5.0,3.531915,106.5,166.0,47.0,59.5,0.026740,3923.406532,4463.097080,3600.0,3900.0,3620,1081,3,5,127,8.139685,6.0,50.368834,45.130,0.333333,0.333333,0.333333,0,0,54.914144,4224,1,0,3.333333,3.0,1.333333,1.0,2.908911,3.0,1.059406,1.0,55097.666667,1473.333333,2899.877193,77.543860,165293.000000,4420.000000,3060.981481,81.851852,18365.888889,491.111111,33058.600000,884.000000,4722.657143,126.285714,9723.117647,260.000000,6887.208333,184.166667,1066.406452,28.516129,33058.600000,884.000000,82646.500000,2210.000000,4349.815789,116.315789,19.000000,52197.789474,0.947368,19.000000,1395.789474,0.947368,16801,20508,23.0,1.958333,1829,19630,26175,37948,7173,6.800000,0.240260,0.125000,2.333333,0.000000,61,3,0,0,1.000000,0.000000,0.000000,0,3,3,50.586667,47.000000,59.50,45.26,6.342583,0,1.000000,0.000000,3,1,3,0,0,0,1.000000,0.000000,0.000000,0.000000,1,0,3,0,0,0,0.000000,1.00000

In [26]:
df.columns[180: ]

Index(['to4_ratio', 'max_toward_type', 'max_toward_ratio',
       'comm_totalFloor_unique', 'comm_totalFloor_len', 'majority',
       'majority_num', 'majority_ratio', 'totalFloor_std', 'mean_year',
       'most_year', 'most_count', 'most_ratio', 'money_per_area',
       'area_type_mean', 'area_type_median', 'area_type_max', 'area_type_min',
       'area_type_num', 'mean_money', 'median_money', 'std_money',
       'mean_area_part_money', 'median_area_part_money', 'std_area_part_money',
       'max_area_part_money', 'min_area_part_money', 'trade_day', 'dayofyear',
       'weekofyear', 'weekday', 'weekend', 'day_count', 'ht_count',
       'ht_encode', 'sale_to_rent_ratio'],
      dtype='object')

In [27]:
train_cols = ['ID', 'area', 'rentType', 'houseFloor', 'totalFloor', 'houseToward',
       'houseDecoration', 'communityName', 'region', 'plate', 'buildYear',
       'saleSecHouseNum', 'subwayStationNum', 'busStationNum',
       'interSchoolNum', 'schoolNum', 'privateSchoolNum', 'hospitalNum',
       'drugStoreNum', 'gymNum', 'bankNum', 'shopNum', 'parkNum', 'mallNum',
       'superMarketNum', 'totalTradeMoney', 'totalTradeArea', 'tradeMeanPrice',
       'tradeSecNum', 'totalNewTradeMoney', 'totalNewTradeArea',
       'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum', 'supplyNewNum',
       'supplyLandNum', 'supplyLandArea', 'tradeLandArea',
       'landTotalPrice', 'landMeanPrice', 'totalWorkers', 'newWorkers',
       'residentPopulation', 'pv', 'uv', 'lookNum', 'room', 'hall', 'bath', 'used_year', 'totalRoom',
       'area_per_room', 'floor_ratio', 'per_pv', 'mean_pv', 'max_pv', 'min_pv',
       'std_pv', 'worker_ratio', 'region_num', 'plate_num', 'community_num',
       'plate_contain', 'community_contain', 'area_type_mean', 'area_type_num', 'area_type_median', 'area_type_max', 'area_type_min', 
       'plate_mean_floor', 'plate_median_floor', 'plate_mean_area',
       'plate_median_area', 'room_ratio', 'hall_ratio', 'bath_ratio', 'room-bath', 'max_type', 'trade_avg',
       'originWorkers', 'area_part', 'tf_part', 'mean_totalRoom', 'median_totalRoom',
       'mean_room', 'median_room', 'plate_mean_totalRoom',
       'plate_median_totalRoom', 'plate_mean_room', 'plate_median_room',
       'drug_per_hosp', 'supermarket_ratio',
       'new_with_subway', 'new_with_bus', 'private_school_ratio',
       'total_school', 'per_subwayStationNum', 'workers_per_subwayStationNum',
       'per_busStationNum', 'workers_per_busStationNum', 'per_interSchoolNum',
       'workers_per_interSchoolNum', 'per_schoolNum', 'workers_per_schoolNum',
       'per_privateSchoolNum', 'workers_per_privateSchoolNum',
       'per_hospitalNum', 'workers_per_hospitalNum', 'per_drugStoreNum',
       'workers_per_drugStoreNum', 'per_gymNum', 'workers_per_gymNum',
       'per_bankNum', 'workers_per_bankNum', 'per_shopNum', 'workers_per_shopNum',
       'per_parkNum', 'workers_per_parkNum', 'per_mallNum',
       'workers_per_mallNum', 'per_superMarketNum',
       'workers_per_superMarketNum', 'sub_div_bus',
       'sub_minus_bus', 'sub_bus_ratio', 'workers_sub_div_bus',
       'workers_sub_minus_bus', 'workers_sub_bus_ratio', 'count_totalFloor',
       'count_tf_part', 'area-year', 'count_buildYear',
       'count_room', 'count_hall', 'count_bath', 'count_area_part',
       '0num', '1num', '2num',
       '0ratio', '1ratio', '2ratio', 'most_type', 'comm_area_len', 'comm_area_kind',
       'comm_mean_area', 'comm_median_area', 'comm_max_area', 'comm_min_area',
       'comm_std_area', 'comm_area_common_num', 'comm_area_unique_ratio',
       'comm_area_common_ratio', 'comm_deco_len', 'comm_deco_kind', '0deco',
       '1deco', '2deco', '3deco', '0deco_ratio', '1deco_ratio', '2deco_ratio',
       '3deco_ratio', 'comm_toward_kind', 'to0', 'to1', 'to2', 'to3', 'to4',
       'to0_ratio', 'to1_ratio', 'to2_ratio', 'to3_ratio', 'to4_ratio', 'max_toward_type', 'max_toward_ratio',
       'comm_totalFloor_unique', 'comm_totalFloor_len', 'majority',
       'majority_num', 'majority_ratio', 'totalFloor_std', 'mean_year',
       'most_year', 'most_count', 'most_ratio', #'money_per_area',
       'area_type_mean', 'area_type_median', 'area_type_max', 'area_type_min',
       'area_type_num', 'mean_money', 'median_money', 'std_money',
       'mean_area_part_money', 'median_area_part_money', 'std_area_part_money',
       'max_area_part_money', 'min_area_part_money', 'trade_day', 'dayofyear',
       'weekofyear', 'weekday', 'weekend', 'day_count', 'ht_count',
       'ht_encode']
        
target = 'tradeMoney'

cate_cols = ['region', 'plate', 'area_part', 'tf_part', 'most_type', 'ht_encode']
for col in cate_cols:
    df[col] = df[col].astype('category')
    
train = df[df['type'] == 1]
test = df[df['type'] == 0]

out_list = [100069036, 100090570, 100150584, 100060167, 100090924, 100028167, 100036712, 100042739, 100100012, 100105848, 100235018, 100021112,
 100080990, 100034012, 100129795, 100124629, 100149709, 100094801, 100016802, 100100923, 100104886, 100172932, 100068168, 100088626,
 100196486, 100002712, 100108480, 100006321, 100300843, 100001807, 100005096, 100213983, 100170024, 100174566, 100049116, 100107239, 100096841,
 100024047, 100307846, 100109091, 100009368, 100030314, 100102076, 100194095, 100012076, 100186256, 100296614, 100024108, 100082168,
 100109566, 100311059, 100024292, 100109705, 100107860, 100160388, 100253280, 100036821, 100148075, 100111792, 100085694, 100127915,
 100050474, 100053400, 100097752, 100038765, 100109503, 100309236, 100173154, 100109779, 100088632, 100303950, 100220858, 100049807,
 100311050, 100018548, 100043022, 100174512, 100094571, 100090724, 100071225, 100048167, 100114674, 100074962, 100262637, 100142019,
 100312210, 100107357, 100077201, 100020172, 100104554, 100186396, 100122337, 100008149, 100266720, 100146578, 100054080, 100218514,
 100108288, 100308771, 100122027]

# train['sample_weights'] = 1
# train.loc[train['ID'].isin(out_list), 'sample_weights'] = 0.1

In [31]:
%%time
train_score_list, val_score_list = [], []
fi_df = pd.DataFrame({'feature': train_cols})
train_sub = train[['ID']]
sub = test[['ID']]

def calc_score(train_, train_t):
    score = 1 - np.sum(np.power(model.predict(train_) - train_t, 2)) / np.sum(np.power(train_t - np.mean(train_t), 2))
    return score

def cus_loss(y_pred, y_true):
    label = y_true.get_label()
    diff = y_pred - label
    grad = 1.5 * diff
    hess = 1.5 * np.ones(y_pred.shape)
    return grad, hess
    
cnt = 0
for seed in [1024, 42, 1]:
    for fold, (train_index, val_index) in enumerate(KFold(n_splits=10, random_state=seed).split(train[train_cols], train['rentType'])):
        print('seed ', seed, 'Fold ', fold)

        l_train = lgb.Dataset(train.iloc[train_index][train_cols], label=train[target].iloc[train_index], categorical_feature=cate_cols)#, weight=train.iloc[train_index]['sample_weights'])
        l_val = lgb.Dataset(train.iloc[val_index][train_cols], label=train[target].iloc[val_index], categorical_feature=cate_cols)#, weight=train.iloc[val_index]['sample_weights'])

        lgb_params = {
            'objective': 'regression_l2', 
            'learning_rate': 0.01, 
            'num_leaves': 45, 
            'min_data_in_leaf': 35, 
            'min_child_samples': 35, 
            'feature_fraction': 0.7, 
            'bagging_freq': 1, 
            'bagging_fraction': 0.85, 
            'bagging_seed': seed, 
            'lambda_l1': 1, 
            'lambda_l2': 2, 
            'metric': 'l2_root'
        }

        model = lgb.train(lgb_params, l_train, valid_sets=[l_train, l_val], fobj=cus_loss, num_boost_round=100000, verbose_eval=0, early_stopping_rounds=200)

        sub['pred'+str(cnt)] = model.predict(test[train_cols])
#         train_sub['pred'+str(cnt)] = model.predict(train[train_cols])
        fi_df['imp'+str(cnt)] = model.feature_importance(importance_type='gain')
        train_ = train.iloc[train_index][train_cols]
        train_t = train[target].iloc[train_index].values
        val_ = train.iloc[val_index][train_cols]
        val_t = train[target].iloc[val_index].values
        train_score = calc_score(train_, train_t)
        val_score = calc_score(val_, val_t)
        train_score_list.append(train_score)
        val_score_list.append(val_score)

        print('train score: ', round(train_score, 6), '\tval score: ', round(val_score, 6))
        cnt += 1
    print('\nseed: ', seed, '\ntrain score: ', np.mean(train_score_list), '\tval score: ', np.mean(val_score_list))
    break
print('\ntrain score: ', np.mean(train_score_list), '\tval score: ', np.mean(val_score_list), '\n')

seed  1024 Fold  0
train score:  0.977605 	val score:  0.943108
seed  1024 Fold  1
train score:  0.973066 	val score:  0.940171
seed  1024 Fold  2
train score:  0.982151 	val score:  0.933232
seed  1024 Fold  3
train score:  0.981251 	val score:  0.939881
seed  1024 Fold  4
train score:  0.98145 	val score:  0.933801
seed  1024 Fold  5
train score:  0.978224 	val score:  0.930757
seed  1024 Fold  6
train score:  0.983118 	val score:  0.937164
seed  1024 Fold  7
train score:  0.985461 	val score:  0.936306
seed  1024 Fold  8
train score:  0.980899 	val score:  0.921084
seed  1024 Fold  9
train score:  0.986627 	val score:  0.948613

seed:  1024 
train score:  0.980985161583 	val score:  0.936411593497

train score:  0.980985161583 	val score:  0.936411593497 

Wall time: 9min 10s


In [37]:
for col in fi_df.columns:
    if 'imp' in col:
        fi_df[col] = fi_df[col].rank()

fi_df['imp'] = np.mean(fi_df[fi_df.columns[1: 9]], axis=1)
fi_df['std'] = np.std(fi_df[fi_df.columns[1: 9]], axis=1)
# plt.figure(figsize=(8, 20))
# # fi_df['imp'].plot(kind='bar',)
# sns.barplot(data=fi_df, x='imp', y='feature')
# plt.show()
fi_df.sort_values('imp', ascending=False)
# fi_df

,feature,imp0,imp1,imp2,imp3,imp4,imp5,imp6,imp7,imp8,imp9,imp,std
1,area,207.0,207.0,207.0,207.0,207.0,207.0,207.0,207.0,207.0,207.0,207.000,0.000000
9,plate,206.0,206.0,206.0,206.0,206.0,206.0,206.0,206.0,206.0,206.0,206.000,0.000000
50,totalRoom,205.0,205.0,205.0,205.0,205.0,205.0,205.0,205.0,205.0,205.0,205.000,0.000000
144,comm_mean_area,201.0,204.0,203.0,204.0,203.0,203.0,204.0,203.0,201.0,203.0,203.125,0.927025
80,area_part,204.0,203.0,202.0,201.0,204.0,204.0,201.0,204.0,204.0,204.0,202.875,1.268611
139,1ratio,202.0,201.0,204.0,203.0,202.0,201.0,202.0,201.0,202.0,201.0,202.000,1.000000
190,mean_money,203.0,202.0,201.0,202.0,201.0,202.0,203.0,202.0,203.0,202.0,202.000,0.707107
87,plate_mean_room,200.0,199.0,199.0,200.0,200.0,200.0,198.0,197.0,200.0,198.0,199.125,1.053269
2,rentType,199.0,200.0,198.0,198.0,199.0,199.0,199.0,200.0,198.0,200.0,199.000,0.707107
206,sale_to_rent_ratio,197.0,198.0,200.0,199.0,198.0,198.0,200.0,199.0,199.0,199.0,198.625,0.992157


train.loc[train['ID'].isin(out_list)]

In [36]:
train_sub['label'] = train['tradeMoney']
train_sub['mean'] = np.mean(train_sub[train_sub.columns[1: 11]], axis=1)
train_sub['median'] = np.median(train_sub[train_sub.columns[1: 11]], axis=1)
train_sub['int_median'] = round(train_sub['median'])
# train_sub['is_co'] = [1 if area <= 20 else 0 for area in train['area']]
# train_sub['median'] = [median*1.1 if is_co == 1 else median for is_co, median in np.array(train_sub[['is_co', 'median']])]
train_sub['mean_error'] = train_sub['label'] - train_sub['mean']
train_sub['median_error'] = train_sub['label'] - train_sub['median']
train_sub['median_perc_error'] = abs(train_sub['median_error'] / train_sub['label'])
train_sub['int_median_error'] = train_sub['label'] - train_sub['int_median']
train_sub['rank'] = abs(train_sub['median_error']).rank(pct=True)
print('mean score: ', 1 - np.sum(np.power(train_sub['mean'] - train_sub['label'], 2)) / np.sum(np.power(train_sub['label'] - np.mean(train_sub['label']), 2)))
print('median score: ', 1 - np.sum(np.power(train_sub['median'] - train_sub['label'], 2)) / np.sum(np.power(train_sub['label'] - np.mean(train_sub['label']), 2)))
print('int_median score: ', 1 - np.sum(np.power(train_sub['int_median'] - train_sub['label'], 2)) / np.sum(np.power(train_sub['label'] - np.mean(train_sub['label']), 2)))
display(train_sub.sort_values('rank', ascending=False).head(50))
out_list = train_sub.sort_values('rank', ascending=False)['ID'].iloc[: 100].tolist()
# display(train_sub.sort_values('median_error'))
# plt.figure(figsize=(8, 8))
# train_sub['mean_error'].hist(bins=100)
# train_sub['median_error'].hist(bins=100)
# plt.scatter(x=train_sub['label'], y=abs(train_sub['median_error']) / train_sub['label'])

mean score:  0.980112863093
median score:  0.982107426778
int_median score:  0.982107462602


,ID,pred0,pred1,pred2,pred3,pred4,pred5,pred6,pred7,pred8,pred9,label,mean,median,int_median,mean_error,median_error,median_perc_error,int_median_error,rank
40300,100069036,6510.175642,6560.979223,7700.755655,8105.326413,7385.553512,6467.507335,8423.007544,8714.187164,7722.978015,1889.634845,13500.0,6948.010535,7543.154583,7543.0,6551.989465,5956.845417,0.441248,5957.0,1.000000
21428,100090570,15053.092815,15060.525118,15866.269011,16554.839085,15822.671025,10233.705619,16395.778344,16831.186961,16037.345180,16413.694232,20000.0,15426.910739,15951.807096,15952.0,4573.089261,4048.192904,0.202410,4048.0,0.999976
946,100150584,3963.846896,5458.814678,6012.982183,6233.957816,5676.857980,5587.670766,6128.301987,6286.326993,5946.778210,5991.693573,10000.0,5728.723108,5969.235892,5969.0,4271.276892,4030.764108,0.403076,4031.0,0.999951
26629,100060167,7975.002936,7786.425481,7212.062054,6820.086726,7642.793120,7785.608863,9531.939294,6693.149880,7100.622171,6902.634300,3500.0,7545.032482,7427.427587,7427.0,-4045.032482,-3927.427587,1.122122,-3927.0,0.999927
21333,100090924,17738.033249,17735.191221,18637.617699,18458.847967,18554.965266,14936.268160,18334.022202,18266.424895,17975.920568,18404.820437,22000.0,17904.211166,18300.223549,18300.0,4095.788834,3699.776451,0.168172,3700.0,0.999902
32315,100028167,16567.154039,17135.391321,17593.964079,17777.647072,17081.346549,16840.534024,18876.837384,13737.714817,17818.300933,17847.820974,21000.0,17127.671119,17364.677700,17365.0,3872.328881,3635.322300,0.173111,3635.0,0.999878
30738,100036712,20129.263468,19964.501413,19876.602964,19893.466066,19865.423957,19917.921697,19920.197355,22485.280513,19660.238863,18674.337208,16500.0,20038.723351,19905.693882,19906.0,-3538.723351,-3405.693882,0.206406,-3406.0,0.999853
29741,100042739,9689.806586,9408.175444,9114.950029,8902.539990,9293.753673,9643.860460,8429.359697,12438.490560,9101.376614,8964.524780,5800.0,9498.683783,9204.351851,9204.0,-3698.683783,-3404.351851,0.586957,-3404.0,0.999829
19856,100100012,6732.600889,6745.502252,6325.399812,5679.535295,9235.630291,6851.373302,5902.369714,5742.513929,6096.391248,6134.381541,2900.0,6544.569827,6229.890676,6230.0,-3644.569827,-3329.890676,1.148238,-3330.0,0.999804
18970,100105848,3477.915890,3540.333383,3659.240940,3823.234158,3224.736770,3528.144528,3795.421009,3868.256440,3682.828864,3767.281954,7000.0,3636.739394,3671.034902,3671.0,3363.260606,3328.965098,0.475566,3329.0,0.999780


In [ ]:
for i in np.linspace(4, 100, 25):
    det = train_sub[(train_sub['label']>= np.percentile(train_sub['label'], i-4.0)) & (train_sub['label'] < np.percentile(train_sub['label'], i))]
    print(i, '\t', np.percentile(train_sub['label'], i-4.0), '-', np.percentile(train_sub['label'], i), \
          round(abs(det['median_error']).mean(), 4), round(abs(det['median_error']).mean() / np.mean(det['label']), 4))

In [30]:
sub1 = sub
sub1
# cv_dict['0.9364_0.9438] = 0.9102

,ID,pred0,pred1,pred2,pred3,pred4,pred5,pred6,pred7,pred8,pred9
40586,100000031,4908.166263,4896.005625,4836.057925,4867.696790,4835.399041,4795.494724,4843.313858,5078.651340,4989.824667,4895.970567
40587,100000061,4019.584748,3992.090757,3797.418612,3907.244699,3956.645786,3985.757680,3990.421004,3832.411294,3926.603046,3735.833921
40588,100000143,6808.075787,6924.232323,6939.682774,6877.533753,6788.888431,6868.822841,6764.247931,6782.954428,6769.339221,7089.749063
40589,100000165,4676.999698,4631.415692,4558.980912,4651.791084,4611.565482,4691.202506,4719.338676,4656.134303,4606.011844,4671.140929
40590,100000181,9056.249425,8986.575957,8988.399587,9038.640537,8982.366808,8956.360308,9051.408174,9025.325559,8686.478905,9266.843691
40591,100000546,2587.394302,2694.045449,2600.346903,2637.277103,2614.276184,2615.091810,2644.396562,2611.159289,2612.269380,2642.314598
40592,100000556,5105.057855,5288.639588,5314.799355,5459.618746,5478.181616,5247.076054,5262.176329,5269.560330,5022.982005,5280.131928
40593,100000575,3546.419979,3511.418569,3547.495964,3541.786067,3589.935804,3524.251364,3570.791016,3524.714732,3489.552536,3485.259289
40594,100000809,4336.915913,4360.526083,4385.376571,4360.254904,4371.835717,4419.972440,4348.783710,4381.877910,4304.753524,4429.187202
40595,100000840,5423.045722,5312.920580,5453.837567,5433.838507,5480.915151,5430.278068,5384.483663,5603.599215,5333.686818,5388.099161


In [32]:
sub2 = sub
sub2.columns = ['ID', 'pred10', 'pred11', 'pred12', 'pred13', 'pred14', 'pred15', 'pred16',
       'pred17', 'pred18', 'pred19']

In [51]:
sub3 = sub
sub3.columns = ['ID', 'pred20', 'pred22', 'pred22', 'pred23', 'pred24', 'pred25', 'pred26',
       'pred27', 'pred28', 'pred29']

In [33]:
sub = sub1.merge(sub2, how='left', on=['ID'])#.merge(sub3, how='left', on=['ID'])
sub

,ID,pred0,pred1,pred2,pred3,pred4,pred5,pred6,pred7,pred8,pred9,pred10,pred11,pred12,pred13,pred14,pred15,pred16,pred17,pred18,pred19
0,100000031,4908.166263,4896.005625,4836.057925,4867.696790,4835.399041,4795.494724,4843.313858,5078.651340,4989.824667,4895.970567,4921.259107,4859.298462,4863.560563,4815.198997,4848.453352,4840.980234,4823.078069,5086.538784,5046.991474,5005.273901
1,100000061,4019.584748,3992.090757,3797.418612,3907.244699,3956.645786,3985.757680,3990.421004,3832.411294,3926.603046,3735.833921,3926.468043,4033.497471,3821.960986,3898.447358,3908.082919,3933.221614,3908.960740,3814.591442,4019.284799,3742.634049
2,100000143,6808.075787,6924.232323,6939.682774,6877.533753,6788.888431,6868.822841,6764.247931,6782.954428,6769.339221,7089.749063,6758.634288,6841.209623,6986.064467,6764.753094,6823.286418,6846.947515,6754.705211,6848.715130,6798.331756,7005.775835
3,100000165,4676.999698,4631.415692,4558.980912,4651.791084,4611.565482,4691.202506,4719.338676,4656.134303,4606.011844,4671.140929,4690.335389,4605.654035,4536.970076,4632.882238,4605.462482,4639.341159,4627.804499,4646.174703,4626.764589,4654.008902
4,100000181,9056.249425,8986.575957,8988.399587,9038.640537,8982.366808,8956.360308,9051.408174,9025.325559,8686.478905,9266.843691,9111.479636,8975.859498,9049.641297,8944.806474,9062.104465,8980.376279,8983.365307,9027.258379,8634.712711,9342.038931
5,100000546,2587.394302,2694.045449,2600.346903,2637.277103,2614.276184,2615.091810,2644.396562,2611.159289,2612.269380,2642.314598,2604.680755,2658.774713,2618.373908,2633.515262,2637.414196,2622.505042,2646.307667,2616.221348,2628.501134,2680.923618
6,100000556,5105.057855,5288.639588,5314.799355,5459.618746,5478.181616,5247.076054,5262.176329,5269.560330,5022.982005,5280.131928,5074.335079,5060.578303,5479.020119,5450.899910,5559.529087,5366.665141,5328.570182,4954.347998,5136.636512,5025.749921
7,100000575,3546.419979,3511.418569,3547.495964,3541.786067,3589.935804,3524.251364,3570.791016,3524.714732,3489.552536,3485.259289,3532.712140,3486.428564,3520.413526,3514.025789,3535.533213,3534.234377,3558.186902,3518.999914,3514.760552,3491.633498
8,100000809,4336.915913,4360.526083,4385.376571,4360.254904,4371.835717,4419.972440,4348.783710,4381.877910,4304.753524,4429.187202,4384.617145,4329.398936,4374.424681,4330.386432,4336.864093,4386.000688,4280.249829,4327.153754,4290.633982,4443.928713
9,100000840,5423.045722,5312.920580,5453.837567,5433.838507,5480.915151,5430.278068,5384.483663,5603.599215,5333.686818,5388.099161,5469.584588,5246.774307,5407.750809,5447.936478,5481.022254,5377.571114,5413.139733,5567.148974,5310.608388,5443.645929


In [34]:
sub['pred'] = np.median(sub[sub.columns[1: 21]], axis=1)
sub[['pred']].to_csv('06_04_0.csv', index=False, header=False)

In [42]:
train.sort_values('tradeMoney', ascending=False)

,ID,area,rentType,houseType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,region,plate,buildYear,saleSecHouseNum,subwayStationNum,busStationNum,interSchoolNum,schoolNum,privateSchoolNum,hospitalNum,drugStoreNum,gymNum,bankNum,shopNum,parkNum,mallNum,superMarketNum,totalTradeMoney,totalTradeArea,tradeMeanPrice,tradeSecNum,totalNewTradeMoney,totalNewTradeArea,tradeNewMeanPrice,tradeNewNum,remainNewNum,supplyNewNum,supplyLandNum,supplyLandArea,tradeLandNum,tradeLandArea,landTotalPrice,landMeanPrice,totalWorkers,newWorkers,residentPopulation,pv,uv,lookNum,tradeTime,tradeMoney,type,room,hall,bath,trade_month,season,totalRoom,area_per_room,floor_ratio,per_pv,mean_pv,max_pv,min_pv,std_pv,worker_ratio,plate_mean,region_mean,plate_median,region_median,region_num,plate_num,community_num,plate_contain,community_contain,plate_mean_floor,plate_median_floor,plate_mean_area,plate_median_area,room_ratio,hall_ratio,bath_ratio,room-bath,max_type,trade_avg,originWorkers,area_part,tf_part,mean_totalRoom,median_totalRoom,mean_room,median_room,plate_mean_totalRoom,plate_median_totalRoom,plate_mean_room,plate_median_room,area_type_mean,area_type_median,area_type_max,area_type_min,area_type_num,per_subwayStationNum,workers_per_subwayStationNum,per_busStationNum,workers_per_busStationNum,per_interSchoolNum,workers_per_interSchoolNum,per_schoolNum,workers_per_schoolNum,per_privateSchoolNum,workers_per_privateSchoolNum,per_hospitalNum,workers_per_hospitalNum,per_drugStoreNum,workers_per_drugStoreNum,per_gymNum,workers_per_gymNum,per_bankNum,workers_per_bankNum,per_shopNum,workers_per_shopNum,per_parkNum,workers_per_parkNum,per_mallNum,workers_per_mallNum,per_superMarketNum,workers_per_superMarketNum,sub_div_bus,sub_minus_bus,sub_bus_ratio,workers_sub_div_bus,workers_sub_minus_bus,workers_sub_bus_ratio,count_totalFloor,count_tf_part,used_year,area-year,count_buildYear,count_room,count_hall,count_bath,count_area_part,drug_per_hosp,supermarket_ratio,new_with_subway,new_with_bus,private_school_ratio,total_school,0num,1num,2num,0ratio,1ratio,2ratio,most_type,comm_area_len,comm_area_kind,comm_mean_area,comm_median_area,comm_max_area,comm_min_area,comm_std_area,comm_area_common_num,comm_area_unique_ratio,comm_area_common_ratio,comm_deco_len,comm_deco_kind,0deco,1deco,2deco,3deco,0deco_ratio,1deco_ratio,2deco_ratio,3deco_ratio,comm_toward_kind,to0,to1,to2,to3,to4,to0_ratio,to1_ratio,to2_ratio,to3_ratio,to4_ratio,max_toward_type,max_toward_ratio,comm_totalFloor_unique,comm_totalFloor_len,majority,majority_num,majority_ratio,totalFloor_std,mean_year,most_year,most_count,most_ratio,money_per_area,mean_money,median_money,std_money,region_type_mean_money,region_type_median_money,mean_area_part_money,median_area_part_money,std_area_part_money,max_area_part_money,min_area_part_money,money_for_room,id_std,id_mean,id_median,plate_type_id_std,plate_type_id_mean,plate_type_id_median,trade_sec_avg_area,trade_sec_avg_money,total_new_num,new_ratio,remain_ratio,new_over_remain,is_supply_new,trade_new_avg_area,trade_new_avg_money,trade_land_ratio,uv_ratio,subwayStationNum_ratio,busStationNum_ratio,interSchoolNum_ratio,schoolNum_ratio,privateSchoolNum_ratio,hospitalNum_ratio,drugStoreNum_ratio,gymNum_ratio,bankNum_ratio,shopNum_ratio,parkNum_ratio,mallNum_ratio,superMarketNum_ratio
37687,100130478,169.000000,1,2室2厅2卫,1,30,3,0,2872,8,27,2004.0,0,7,74,1,72,13,4,37,38,50,306,5,11,63,170850000,2475.07,69028.35071,38,1918864532,20376,94172.77837,180,72,0,0,0.00,0,0.00,0,0.000000,349715,0,266779,10115.0,1979.0,0,2018/9/30,25000.0,1,2,2,2,9,3,6,28.166667,15.0,5.111167,6047.0,10115.0,1979.0,4068.0,1.310879,11037.636364,8940.446309,11000.0,8600.0,1323,373,4,2,76,21.506702,24.0,69.497020,61.360,0.333333,0.333333,0.333333,0,0,63.463333,349715,5,5,5.500000,6.0,2.000000,2.0,6.067797,6.0,2.677966,3.0,11981.361063,12500.0,25000.0,2000.0,1798,33347.375000,43714.375000,3557.053333,4662.866667,133389.500000,174857.500000,3654.506849,4790.616438,19055.642857,24979.642857,53355.8

In [43]:
test.sort_values('area', ascending=False)

,ID,area,rentType,houseType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,region,plate,buildYear,saleSecHouseNum,subwayStationNum,busStationNum,interSchoolNum,schoolNum,privateSchoolNum,hospitalNum,drugStoreNum,gymNum,bankNum,shopNum,parkNum,mallNum,superMarketNum,totalTradeMoney,totalTradeArea,tradeMeanPrice,tradeSecNum,totalNewTradeMoney,totalNewTradeArea,tradeNewMeanPrice,tradeNewNum,remainNewNum,supplyNewNum,supplyLandNum,supplyLandArea,tradeLandNum,tradeLandArea,landTotalPrice,landMeanPrice,totalWorkers,newWorkers,residentPopulation,pv,uv,lookNum,tradeTime,tradeMoney,type,room,hall,bath,trade_month,season,totalRoom,area_per_room,floor_ratio,per_pv,mean_pv,max_pv,min_pv,std_pv,worker_ratio,plate_mean,region_mean,plate_median,region_median,region_num,plate_num,community_num,plate_contain,community_contain,plate_mean_floor,plate_median_floor,plate_mean_area,plate_median_area,room_ratio,hall_ratio,bath_ratio,room-bath,max_type,trade_avg,originWorkers,area_part,tf_part,mean_totalRoom,median_totalRoom,mean_room,median_room,plate_mean_totalRoom,plate_median_totalRoom,plate_mean_room,plate_median_room,area_type_mean,area_type_median,area_type_max,area_type_min,area_type_num,per_subwayStationNum,workers_per_subwayStationNum,per_busStationNum,workers_per_busStationNum,per_interSchoolNum,workers_per_interSchoolNum,per_schoolNum,workers_per_schoolNum,per_privateSchoolNum,workers_per_privateSchoolNum,per_hospitalNum,workers_per_hospitalNum,per_drugStoreNum,workers_per_drugStoreNum,per_gymNum,workers_per_gymNum,per_bankNum,workers_per_bankNum,per_shopNum,workers_per_shopNum,per_parkNum,workers_per_parkNum,per_mallNum,workers_per_mallNum,per_superMarketNum,workers_per_superMarketNum,sub_div_bus,sub_minus_bus,sub_bus_ratio,workers_sub_div_bus,workers_sub_minus_bus,workers_sub_bus_ratio,count_totalFloor,count_tf_part,used_year,area-year,count_buildYear,count_room,count_hall,count_bath,count_area_part,drug_per_hosp,supermarket_ratio,new_with_subway,new_with_bus,private_school_ratio,total_school,0num,1num,2num,0ratio,1ratio,2ratio,most_type,comm_area_len,comm_area_kind,comm_mean_area,comm_median_area,comm_max_area,comm_min_area,comm_std_area,comm_area_common_num,comm_area_unique_ratio,comm_area_common_ratio,comm_deco_len,comm_deco_kind,0deco,1deco,2deco,3deco,0deco_ratio,1deco_ratio,2deco_ratio,3deco_ratio,comm_toward_kind,to0,to1,to2,to3,to4,to0_ratio,to1_ratio,to2_ratio,to3_ratio,to4_ratio,max_toward_type,max_toward_ratio,comm_totalFloor_unique,comm_totalFloor_len,majority,majority_num,majority_ratio,totalFloor_std,mean_year,most_year,most_count,most_ratio,money_per_area,mean_money,median_money,std_money,region_type_mean_money,region_type_median_money,mean_area_part_money,median_area_part_money,std_area_part_money,max_area_part_money,min_area_part_money,money_for_room,id_std,id_mean,id_median,plate_type_id_std,plate_type_id_mean,plate_type_id_median,trade_sec_avg_area,trade_sec_avg_money,total_new_num,new_ratio,remain_ratio,new_over_remain,is_supply_new,trade_new_avg_area,trade_new_avg_money,trade_land_ratio,uv_ratio,subwayStationNum_ratio,busStationNum_ratio,interSchoolNum_ratio,schoolNum_ratio,privateSchoolNum_ratio,hospitalNum_ratio,drugStoreNum_ratio,gymNum_ratio,bankNum_ratio,shopNum_ratio,parkNum_ratio,mallNum_ratio,superMarketNum_ratio
41785,100122404,150.00,0,4室2厅3卫,2,5,1,3,3418,11,15,2011.0,0,2,98,0,9,1,0,13,1,7,84,5,6,16,435800000,17444.70,24981.79963,182,455921570,10278,44358.97743,61,640,0,0,0.00,0,0.00,0,0.000000,3725,0,63153,13001.0,1265.0,0,2018/3/10,NaN,0,4,2,3,3,1,9,16.666667,4.0,10.277470,7133.0,13001.0,1265.0,5868.0,0.058984,3854.400000,5257.065817,3500.0,4300.0,740,248,2,3,31,15.012097,18.0,77.740517,81.000,0.444444,0.222222,0.333333,1,0,95.326230,3725,5,0,8.500000,8.5,4.000000,4.0,7.535714,7.5,3.428571,3.0,6770.694098,6500.0,13999.0,2000.0,5389,21051.000000,1241.666667,637.909091,37.626263,63153.000000,3725.000000,6315.300000,372.500000,31576.500000,1862.500000,63153.000000,3725.000000,4510.928571,26